PREPARATION FOR BLASTX

In [ ]:
#Code to enable the BLAST of the assembled contigs against our MaRNAV1 RDRP dataset made up of Accessions MN698826-MN698829.

import glob
import os
AssembledFastas = glob.glob("/home/lj1893/LabProjectBiocomp/fasta/assembled_fastas/*")
print("yes")
count = 0
for file in AssembledFastas:
    print("yes")
    print (count) 
    part1 = file.split("/")[-1]
    part2 = part1.split(".fasta")[0]
    output_file = f"/home/lj1893/LabProjectBiocomp/fasta/BLAST_Database/RDRPBlastData/{part2}.out"
    print(part2)
    blast_command = f"blastx -query {file} -db '/home/lj1893/LabProjectBiocomp/fasta/BLAST_Database/DiagnosticProteins/RDRP/RDRP.fasta' -outfmt 6 > {output_file}" 
    os.system(blast_command)
    count += 1

In [ ]:
#Compiled the RDRP sequences and formulated a BLAST database that we can use to compare to our datasets

%%bash
makeblastdb -in ~/LabProjectBiocomp/fasta/BLAST_Database/DiagnosticProteins/RDRP/RDRP.fasta -dbtype prot

In [ ]:
#This is the blast funciton that we have generated as an experiment to compare the known RDRP sequence against a single assembled fasta file
#Output was directed to a file (.out)

%%bash
blastx -query ~/LabProjectBiocomp/fasta/assembled_fastas/ERR1717087_rnaspades.fasta -db ~/LabProjectBiocomp/fasta/BLAST_Database/DiagnosticProteins/RDRP/RDRP.fasta -outfmt 6 > ~/LabProjectBiocomp/fasta/BLAST_Database/BLASTTesting.out


In [ ]:
#Code to tabulate the results of this single test

cols = ['query',
       'target',
       'percent_identity',
       'alignment_length',
       'n_mismatches',
       'n_gap_opens',
       'start_pos_query',
       'end_pos_query',
       'start_pos_target',
        'end_pos_target',
       'evalue',
       'bit_score']

import pandas as pd
tab = pd.read_csv("~/LabProjectBiocomp/fasta/BLAST_Database/BLASTTesting.out", sep = "\t", names=cols)


In [ ]:
#Code to tabulate only the rows that achieve a bit score of over 500
[tab['bit_score'] > 500]
positive_tab = tab[tab['bit_score'] > 500]
positive_tab

len(positive_tab) 

In [ ]:
#Code to plot a table of the contigs with bitscores > 500, count number of positive contigs, place all positive contigs in a file

countPositive = 0
counutNegative = 0
AllPositiveTable = pd.DataFrame()
output = open("blast_output/outcome.txt, "w")

for file in glob.glob("file destinatinon"):
    tab = pd.read_csv(file, names = cols, sep "\t")
    tab['file'] = file
    positive_tab = tab[tab['bitscore'] > 500]
    if len(positive_tab) >0:
        countPositive = countPositive + 1
        status = '+'
    else:
        countNegative = countNegative + 1
        status = '1'
    output.write(f"{file}\t{status}\n")
    AllPositveTable = pd.concat([AllPositiveTable, positive_tab]) 
output.close()

AllPositiveTable.to_csv('blast_output/combined_positives.txt', sep = "\t", index = None)

RUNNING THE BLASTX AND PROCESSING RESULTS FOR RDRP (SEGMENT 1)

In [ ]:
#Code to run the full BLAST of all assembled contigs against the RDRP BlAST dataset

import glob
import os
AssembledFastas = glob.glob("/home/lj1893/LabProjectBiocomp/fasta/assembled_fastas/*")
print("yes")
count = 0
for file in AssembledFastas:
    print("yes")
    print (count) 
    part1 = file.split("/")[-1]
    part2 = part1.split(".fasta")[0]
    output_file = f"/home/lj1893/LabProjectBiocomp/fasta/BLAST_Database/RDRPBlastData/{part2}.out"
    print(part2)
    blast_command = f"blastx -query {file} -db '/home/lj1893/LabProjectBiocomp/fasta/BLAST_Database/DiagnosticProteins/RDRP/RDRP.fasta' -outfmt 6 > {output_file}" 
    os.system(blast_command)
    count += 1

In [ ]:
#Code to process the results of the BLASTX against RDRP
#Formulating a table selecting for contigs with a bitscore > 500
#Generating an output file with the list of positive contigs called 'CombinedPositives.txt"

import pandas as pd
import glob
import os

cols = ['query',
       'target',
       'percent_identity',
       'alignment_length',
       'n_mismatches',
       'n_gap_opens',
       'start_pos_query',
       'end_pos_query',
       'start_pos_target',
        'end_pos_target',
       'evalue',
       'bit_score']

count_ViralPositive = 0
count_ViralNegative = 0
with open("/home/lj1893/LabProjectBiocomp/fasta/BLAST_Database/RDRPBlastData/Analysis.txt", 'w') as output:
    
    MainTable = pd.DataFrame()
    for file in glob.glob("/home/lj1893/LabProjectBiocomp/fasta/BLAST_Database/RDRPBlastData/*out"):
        tab = pd.read_csv(file, names=cols, sep="\t")
        tab['file'] = file 
        ViralPositiveTab = tab[tab['bit_score'] > 500] 
        if len(ViralPositiveTab) >0:
                count_ViralPositive = count_ViralPositive + 1 
                status = '+'
        else:
                count_ViralNegative = count_ViralNegative + 1 
                status = '-'

        print("ViralPositive: {count_ViralPositive} \nViralNegative: {count_ViralNegative}")
        print(count_ViralPositive)

        

        output.write(f"{file}\t{status}\n") 
        MainTable = pd.concat([MainTable, ViralPositiveTab])

MainTable.to_csv("/home/lj1893/LabProjectBiocomp/fasta/BLAST_Database/RDRPBlastData/CombinedPositives.txt", sep="\t", index=None) 

In [ ]:
#After screening based on bitscore > 500 to create CombinedPositives.txt we screened again for 95% sequence similarity 
#Formed a file called 'RDRPPositiveContigs.fasta"

import pandas as pd
import pyfaidx as f
import os

PositiveNinetyFive = tab[tab["percent_identity"] > 95]

gb = PositiveNinetyFive.groupby('file') 
RDRPPositiveContigs = open("/home/lj1893/LabProjectBiocomp/fasta/VirusSequenceAssembly/RDRPPositiveContigs/RDRPPositiveContigs.fasta", "w")
gb.groups  

for group in gb.groups:  
    one = group.split("/")[-1]
    two = one.split(".out")[0]
    fasta = f"/home/lj1893/LabProjectBiocomp/fasta/assembled_fastas/{two}.fasta"
    FastaFileContents = f.Fasta(fasta) 
    small_table = gb.get_group(group)
    for SequenceName in small_table['query']:
   
        sequence = FastaFileContents[SequenceName][:]
        s = sequence.seq
        RDRPPositiveContigs.write(f">{two}_{SequenceName}\n{s}\n")
        print(len(s))
RDRPPositiveContigs.close()


RUNNING THE BLASTX AND PROCESSING RESULTS FOR HYPOTHETICAL PROTEIN (SEGMENT 2)

In [ ]:
#Code to run the full BLAST of all assembled contigs against the Hypothetical Protein BlAST dataset

import glob
import os
AssembledFastas = glob.glob("/home/lj1893/LabProjectBiocomp/fasta/assembled_fastas/*")
print("yes")
count = 0
for file in AssembledFastas:
    print("yes")
    print (count) 
    part1 = file.split("/")[-1]
    
    part2 = part1.split(".fasta")[0]
    output_file = f"/home/lj1893/LabProjectBiocomp/fasta/BLAST_Database/HypotheticalProteinBlastData/{part2}.out"
    print(part2)
    blast_command = f"blastx -query {file} -db /home/lj1893/LabProjectBiocomp/fasta/BLAST_Database/DiagnosticProteins/HypotheticalProteins/HypotheticalProtein.fasta -outfmt 6 > {output_file}" 
    os.system(blast_command)
    count += 1

In [ ]:
#Code to process the results of the BLASTX against Hypothetical Protein
#Formulating a table selecting for contigs with a bitscore > 500
#Generating an output file with the list of positive contigs called 'CombinedPositives.txt"

import pandas as pd
import glob
import os

cols = ['query',
       'target',
       'percent_identity',
       'alignment_length',
       'n_mismatches',
       'n_gap_opens',
       'start_pos_query',
       'end_pos_query',
       'start_pos_target',
        'end_pos_target',
       'evalue',
       'bit_score']

count_ViralPositive = 0
count_ViralNegative = 0
with open("/home/lj1893/LabProjectBiocomp/fasta/BLAST_Database/HypotheticalProteinBlastData/Analysis.txt", 'w') as output: 

    MainTable = pd.DataFrame()
    for file in glob.glob("/home/lj1893/LabProjectBiocomp/fasta/BLAST_Database/HypotheticalProteinBlastData/*out"):
        tab = pd.read_csv(file, names=cols, sep="\t")
        tab['file'] = file 
        ViralPositiveTab = tab[tab['bit_score'] > 200]
        if len(ViralPositiveTab) >0:
                count_ViralPositive = count_ViralPositive + 1 
                status = '+'
        else:
                count_ViralNegative = count_ViralNegative + 1 
                status = '-'

        print("ViralPositive: {count_ViralPositive} \nViralNegative: {count_ViralNegative}")
        print(count_ViralPositive)

        

        output.write(f"{file}\t{status}\n") 
        MainTable = pd.concat([MainTable, ViralPositiveTab])

MainTable.to_csv("/home/lj1893/LabProjectBiocomp/fasta/BLAST_Database/HypotheticalProteinBlastData/CombinedPositives.txt", sep="\t", index=None)

In [ ]:
#After screening based on bitscore > 500 to create CombinedPositives.txt we screened again for 95% sequence similarity 
#Formed a file called 'HypotheticalProteinPositiveContigs.fasta"

PositiveNinetyFive = tab[tab["percent_identity"] > 95]

gb = PositiveNinetyFive.groupby('file')
HypotheticalProteinPositiveContigs = open("/home/lj1893/LabProjectBiocomp/fasta/VirusSequenceAssembly/HypotheticalProteinPositiveContigs/HypotheticalProteinPositiveContigs.fasta", "w")
gb.groups  


for group in gb.groups:  
    one = group.split("/")[-1]
    two = one.split(".out")[0]
    fasta = f"/home/lj1893/LabProjectBiocomp/fasta/assembled_fastas/{two}.fasta"
    FastaFileContents = f.Fasta(fasta) 
    small_table = gb.get_group(group) 
    for SequenceName in small_table['query']: 
        
        sequence = FastaFileContents[SequenceName][:]
        s = sequence.seq
        HypotheticalProteinPositiveContigs.write(f">{two}_{SequenceName}\n{s}\n")
        print(len(s))
HypotheticalProteinPositiveContigs.close()
        